In [50]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
df=pd.read_csv('train1.csv')
df.head()

,Webpage_id,Domain,Url,Tag,Vector
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,news,marketing tecfidera gilenya and aubagio s 3 wa...
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,news,pharma novo equipped to weather storm u s diab...
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,news,pharma another exec departs troubled endo and ...
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,news,pharma teva buy biosim specialist celltrion it...
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,news,marketing actress marissa tomei partners aller...


In [51]:
df.shape

(53447, 5)

In [52]:
from sklearn.utils import shuffle
final=shuffle(df)

In [60]:
final.fillna('a', inplace=True)

In [61]:
def filter_num(s):
    s = s.split()
    s = list(filter(str.isalpha, s))
    return ' '.join(s)

In [63]:
final.Vector = final.Vector.map(filter_num)

In [64]:
final.head()

,Webpage_id,Domain,Url,Tag,Vector
31959,47703,forums.thebump.com,https://forums.thebump.com/discussion/12460811...,forum,discussion newbie intro
53225,79014,medicine.uiowa.edu,https://medicine.uiowa.edu/anesthesia/profile/...,profile,anesthesia profile jonathan simmons
42602,63972,career.bayer.com,https://career.bayer.com/en/career/working-at-...,others,en career working at bayer graduates internati...
13168,19659,jla.sagepub.com,http://journals.sagepub.com/doi/full/10.1177/2...,publication,doi full
22106,33409,news.mit.edu,http://news.mit.edu/department/brain-and-cogni...,others,department brain and cognitive sciences


In [54]:
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bhargava\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{"won't", 'mustn', 'i', 'was', 'so', "needn't", 'hers', "she's", 'me', 'had', 'aren', "isn't", 'once', 'more', "don't", 'between', 'at', 'hasn', 'no', 'he', "shan't", 'while', 'off', 'other', 't', 'y', 'until', "doesn't", "weren't", 'our', 'above', 'few', 'where', 'most', 'yourselves', 'in', 'both', 'ma', 'be', 'do', 'whom', 'during', 'down', 'didn', 'such', 'further', 'each', 'by', 'any', 're', 'couldn', 'have', 'weren', 'their', 'myself', "you're", 'hadn', 'about', 'been', 'themselves', 'has', 'but', 'how', 'nor', 'too', 'all', 'doesn', 'ours', 'through', 'a', 'what', "didn't", 'own', 'them', 'we', 'and', 'into', 'from', 'mightn', 'now', 'ain', 'does', 'for', 'an', "mustn't", 'll', 'did', 's', 'here', 'shouldn', 've', 'if', 'over', 'there', 'him', 'same', "shouldn't", 'itself', "mightn't", 'wouldn', "you'll", 'that', 'her',

In [65]:
i=0
str1=' '
final_string=[]
#all_positive_words=[] # store words from +ve reviews here
#all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Vector'].values:
    filtered_sentence=[]
    #print(sent);
     # remove HTMl tags
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [66]:
final['CleanedText']=final_string

In [67]:
final.head(3)

,Webpage_id,Domain,Url,Tag,Vector,CleanedText
31959,47703,forums.thebump.com,https://forums.thebump.com/discussion/12460811...,forum,discussion newbie intro,b'discuss newbi intro'
53225,79014,medicine.uiowa.edu,https://medicine.uiowa.edu/anesthesia/profile/...,profile,anesthesia profile jonathan simmons,b'anesthesia profil jonathan simmon'
42602,63972,career.bayer.com,https://career.bayer.com/en/career/working-at-...,others,en career working at bayer graduates internati...,b'career work bayer graduat international oper...


In [68]:
final.shape

(53447, 6)

In [69]:
final.size

320682

In [71]:
#BoW
#without unigrams and bigrams
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(final['CleanedText'].values)

In [72]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [73]:
final_counts.get_shape()

(53447, 17756)

In [74]:
final.shape

(53447, 6)

In [75]:
final.head()

,Webpage_id,Domain,Url,Tag,Vector,CleanedText
31959,47703,forums.thebump.com,https://forums.thebump.com/discussion/12460811...,forum,discussion newbie intro,b'discuss newbi intro'
53225,79014,medicine.uiowa.edu,https://medicine.uiowa.edu/anesthesia/profile/...,profile,anesthesia profile jonathan simmons,b'anesthesia profil jonathan simmon'
42602,63972,career.bayer.com,https://career.bayer.com/en/career/working-at-...,others,en career working at bayer graduates internati...,b'career work bayer graduat international oper...
13168,19659,jla.sagepub.com,http://journals.sagepub.com/doi/full/10.1177/2...,publication,doi full,b'doi full'
22106,33409,news.mit.edu,http://news.mit.edu/department/brain-and-cogni...,others,department brain and cognitive sciences,b'depart brain cognit scienc'


In [76]:
#adding a column named WordCount which tells us the number of legitimate words in the review text
final['WordCount']= final['CleanedText'].values
final['WordCount'] = final['WordCount'].apply(lambda i: len(i.split()))

In [77]:
final.head()

,Webpage_id,Domain,Url,Tag,Vector,CleanedText,WordCount
31959,47703,forums.thebump.com,https://forums.thebump.com/discussion/12460811...,forum,discussion newbie intro,b'discuss newbi intro',3
53225,79014,medicine.uiowa.edu,https://medicine.uiowa.edu/anesthesia/profile/...,profile,anesthesia profile jonathan simmons,b'anesthesia profil jonathan simmon',4
42602,63972,career.bayer.com,https://career.bayer.com/en/career/working-at-...,others,en career working at bayer graduates internati...,b'career work bayer graduat international oper...,10
13168,19659,jla.sagepub.com,http://journals.sagepub.com/doi/full/10.1177/2...,publication,doi full,b'doi full',2
22106,33409,news.mit.edu,http://news.mit.edu/department/brain-and-cogni...,others,department brain and cognitive sciences,b'depart brain cognit scienc',4


In [78]:
final_counts

<53447x17756 sparse matrix of type '<class 'numpy.int64'>'
	with 188308 stored elements in Compressed Sparse Row format>

In [79]:
y=final['Tag'].values

In [80]:
y.shape

(53447,)

In [82]:
dff=pd.read_csv('test1.csv')

In [83]:
dff.head()

,Webpage_id,Domain,Url,Vector,Length,Alpha,Num,Alnum
0,31,isrctn.com,http://www.isrctn.com/ISRCTN57801413,isrctn57801413,14,0,0,1
1,32,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,ctr search trial 2006 006214 16 gb,34,4,3,0
2,33,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,ctr search trial 2006 004265 34 lt,34,4,3,0
3,34,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,ctr search trial 2010 022183 12 it,34,4,3,0
4,35,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,ctr search trial 2010 021349 36 nl,34,4,3,0


In [84]:
from sklearn.utils import shuffle
final_test=shuffle(dff)

In [85]:
final_test.head()

,Webpage_id,Domain,Url,Vector,Length,Alpha,Num,Alnum
23602,73426,heb.sagepub.com,http://journals.sagepub.com/doi/full/10.1177/1...,doi full 10 1177 1090198110382502,33,2,3,0
16292,48676,unpregnantchicken.com,http://unpregnantchicken.com/2016/02/micro-pos...,2016 02 micro post breastfeeding,32,3,2,0
18000,54352,www.kjronline.org,https://www.kjronline.org/index.php?body=archive,index php,9,2,0,0
15980,48240,patient.info,https://patient.info/forums/discuss/early-preg...,forums discuss early pregnancy symptoms or end...,63,7,1,0
13072,38767,biology.mit.edu,https://biology.mit.edu/people/alan_grossman,people alan_grossman,20,1,0,0


In [87]:
final_test.fillna('a', inplace=True)

In [88]:
def filter_num(s):
    s = s.split()
    s = list(filter(str.isalpha, s))
    return ' '.join(s)

In [89]:
final_test.Vector = final_test.Vector.map(filter_num)

In [90]:
final_test.head()

,Webpage_id,Domain,Url,Vector,Length,Alpha,Num,Alnum
23602,73426,heb.sagepub.com,http://journals.sagepub.com/doi/full/10.1177/1...,doi full,33,2,3,0
16292,48676,unpregnantchicken.com,http://unpregnantchicken.com/2016/02/micro-pos...,micro post breastfeeding,32,3,2,0
18000,54352,www.kjronline.org,https://www.kjronline.org/index.php?body=archive,index php,9,2,0,0
15980,48240,patient.info,https://patient.info/forums/discuss/early-preg...,forums discuss early pregnancy symptoms or end...,63,7,1,0
13072,38767,biology.mit.edu,https://biology.mit.edu/people/alan_grossman,people,20,1,0,0


In [91]:
i=0
str1=' '
final_string=[]
#all_positive_words=[] # store words from +ve reviews here
#all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final_test['Vector'].values:
    filtered_sentence=[]
    #print(sent);
     # remove HTMl tags
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [92]:
final_test['CleanedText']=final_string

# Creating features:
length of the string.
number of alpha words in string.
number of numeric words in the string.
hashed domain name.
presence of some words.
may be stemming.

In [93]:
final_test.head()

,Webpage_id,Domain,Url,Vector,Length,Alpha,Num,Alnum,CleanedText
23602,73426,heb.sagepub.com,http://journals.sagepub.com/doi/full/10.1177/1...,doi full,33,2,3,0,b'doi full'
16292,48676,unpregnantchicken.com,http://unpregnantchicken.com/2016/02/micro-pos...,micro post breastfeeding,32,3,2,0,b'micro post breastfeed'
18000,54352,www.kjronline.org,https://www.kjronline.org/index.php?body=archive,index php,9,2,0,0,b'index php'
15980,48240,patient.info,https://patient.info/forums/discuss/early-preg...,forums discuss early pregnancy symptoms or end...,63,7,1,0,b'forum discuss earli pregnanc symptom endomet...
13072,38767,biology.mit.edu,https://biology.mit.edu/people/alan_grossman,people,20,1,0,0,b'peopl'


In [94]:

#Dropped webpage ID as that shouldn't be used as a feature
count_vect = CountVectorizer() #in scikit-learn
final_countstest = count_vect.fit_transform(final_test['CleanedText'].values)

In [95]:
final_countstest.get_shape()

(25787, 10351)

In [ ]:
#A lot of data cleaning is done till now
import pandas as pd
ht=pd.read_csv('html_data.csv')

In [ ]:
ht.head()

In [ ]:
import time

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from bs4 import BeautifulSoup
import re

In [ ]:
train = pd.read_csv('train1.csv')
test = pd.read_csv('test1.csv')

In [ ]:
tree = DecisionTreeClassifier(max_depth=100, criterion='gini')
tree.fit(X, y)
pred = tree.predict(X_test)

In [ ]:
sample_sub = pd.read_csv('sample_submission.csv')

In [ ]:
sample_sub.Tag = pred
sample_sub.to_csv('decisiontreedepth100.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest=RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=50)
forest.fit(X,y)

In [ ]:
pred1=forest.predict(X_test)

In [ ]:
sample_sub.Tag = pred1
sample_sub.to_csv('random forest1.csv', index=False)